# Pytorch Quickstart
- URL from https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html

In [2]:
# 라이브러리 도입(?)부
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# 공개 데이터셋에서 학습과 시험용 데이터를 내려받아보자

# 학습용 데이터
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 시험용 데이터
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Dataset을 DataLoader의 인자로 전달합니다.
- 함수 f(x)의 'x'같은 역할인 거 같다.  

이는 데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원합니다. 여기서는 배치 크기(batch size)를 64로 정의합니다.
- 수업 때 듣기를 이 batch size가 컴퓨터 성능에 지대한 영향을 받는다고 함.

즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환합니다.

In [4]:
batch_size = 64

# 데이터로더를 생성하는 부분
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


PyTorch에서 신경망 모델은 nn.Module 을 상속받는 클래스(class)를 생성하여 정의합니다. __init__ 함수에서 신경망의 계층(layer)들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정합니다. 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate)합니다.
- 이번 부분에서는 내 local 환경이 에러가 너무 넘쳐나고 GPU가 재정상 갖추어져있지 않으므로 Google Colab에 탑승해서 해볼 것이다.

In [5]:
# 이 연산을 돌릴 장치를 GPU 또는 CPU로 설정하자
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"이 연산에서는 {device} 장치를 사용할 거예요!")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

이 연산에서는 cuda 장치를 사용할 거예요!
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델 매개변수 최적화하기
- 모델을 학습하려면 손실 함수(loss function)와 옵티마이저(optimizer, 최적화 도구??)가 필요하다.

In [6]:
loss_fn = nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # optimizer

각 학습 단계에서 모델은 batch로 제공되는 학습 데이터셋에 대한 예측을 하고 예측 오류를 역전파해서 모델의 매개변수를 조정한다.

In [8]:
# 데이터셋으로 학습시키는 함수
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

모델이 학습하고 있는지 확인하기 위해 시험용 데이터셋으로 모델의 성능을 확인해보자.

In [9]:
# 시험해보는(=모델의 성능을 확인해보는) 함수
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 단계는 여러번의 반복 단계(=epochs)를 거쳐서 수행된다. 각 epoch에서는 모델은 더 나은 예측을 하려고 매개변수를 학습한다. 각 epoch마다 모델의 정확도(accuracy)와 손실값(loss)을 출력한다. epoch마다 정확도가 증가하고 손실이 감소하는 것을 봐보자.

In [10]:
epochs = 5 # 반복 횟수 : 5회
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300278  [    0/60000]
loss: 2.287895  [ 6400/60000]
loss: 2.268839  [12800/60000]
loss: 2.266562  [19200/60000]
loss: 2.246165  [25600/60000]
loss: 2.209774  [32000/60000]
loss: 2.219397  [38400/60000]
loss: 2.174034  [44800/60000]
loss: 2.187366  [51200/60000]
loss: 2.141587  [57600/60000]
Test Error: 
 Accuracy: 50.3%, Avg loss: 2.139336 

Epoch 2
-------------------------------
loss: 2.149992  [    0/60000]
loss: 2.137578  [ 6400/60000]
loss: 2.074971  [12800/60000]
loss: 2.096526  [19200/60000]
loss: 2.044163  [25600/60000]
loss: 1.974463  [32000/60000]
loss: 2.001958  [38400/60000]
loss: 1.910201  [44800/60000]
loss: 1.933342  [51200/60000]
loss: 1.843177  [57600/60000]
Test Error: 
 Accuracy: 60.1%, Avg loss: 1.846890 

Epoch 3
-------------------------------
loss: 1.883062  [    0/60000]
loss: 1.851486  [ 6400/60000]
loss: 1.726904  [12800/60000]
loss: 1.773733  [19200/60000]
loss: 1.682346  [25600/60000]
loss: 1.618863  [32000/600

모델을 저장하는 일반적인 방법은 모델의 매개변수들을 포함해서 내부 상태 사전(internal state dictionary)을 직렬화(serialize)하는 것이다.
- 튜토리얼 문서의 의미가 좀 이해되지 않아 추가로 적어본다. internal  state dictionary는 좀 더 찾아보니 학습할 수 있는 매개변수들은 모델의 매개변수에도 저장된다는 의미인 것으로 보인다. 다만, 참고한 문서(https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html#:~:text=A%20state_dict%20is%20an%20integral,to%20PyTorch%20models%20and%20optimizers.)에서는 state dict의 의미에 대해서만 적어두어 좀 더 찾아볼 필요가 있어보인다.

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth", "모델이 저장되었습니다!")

Saved PyTorch Model State to model.pth 모델이 저장되었습니다!


모델을 불러오려면 모델 구조를 다시 만들고 위에서 저장했던 것을 불러오면 된다.

In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

이제는 저장하고 불러온 모델을 사용해서 예측을 해볼 수 있다. 아까 학습+시험용으로 불러온 패션 아이템 데이터셋에 대해서 해보자.

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
] # 패션 아이템 데이터 label들

model.eval() # 평가 시작!
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
